[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bobleesj/quantem.widget/blob/main/notebooks/bin/bin_simple.ipynb)

# Bin - Quick Demo

Interactive calibration-aware 4D-STEM binning with BF/ADF quality checks.
Uses synthetic SrTiO3 crystal data with realistic Bragg spots.

In [11]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: ANYWIDGET_HMR=1


In [12]:
import numpy as np
from quantem.widget import Bin

In [13]:
# --- Synthetic 4D-STEM: SrTiO3 crystal with Bragg spots ---
import math

scan_rows, scan_cols = 32, 32
det_rows, det_cols = 96, 96
pixel_size = 2.39   # Angstrom/px
k_pixel_size = 0.46  # mrad/px

# Detector coordinate grid
ky = np.arange(det_rows, dtype=np.float32) - det_rows / 2
kx = np.arange(det_cols, dtype=np.float32) - det_cols / 2
KY, KX = np.meshgrid(ky, kx, indexing='ij')
kr2 = KY**2 + KX**2

# Central beam (bright field disk)
central_beam = np.exp(-kr2 / (2 * 12.0**2))

# Bragg spots — cubic perovskite reciprocal lattice
lattice_a = 20.0
bragg_spots = np.zeros_like(central_beam)
for h in range(-2, 3):
    for k in range(-2, 3):
        if h == 0 and k == 0:
            continue
        dist2 = (KY - h * lattice_a)**2 + (KX - k * lattice_a)**2
        intensity = 0.3 * math.exp(-0.1 * (h**2 + k**2))
        bragg_spots += intensity * np.exp(-dist2 / (2 * 2.5**2))

base_dp = central_beam + bragg_spots

# Scan-position-dependent intensity (atom column contrast)
sy = np.linspace(-1, 1, scan_rows, dtype=np.float32)
sx = np.linspace(-1, 1, scan_cols, dtype=np.float32)
SY, SX = np.meshgrid(sy, sx, indexing='ij')
atom_contrast = 1.0 + 0.15 * np.sin(2 * np.pi * SY * 4) * np.sin(2 * np.pi * SX * 4)

# Build 4D with Poisson noise
data4d = base_dp[None, None] * atom_contrast[:, :, None, None]
rng = np.random.default_rng(42)
data = rng.poisson(np.clip(data4d * 100, 0, None)).astype(np.float32)

print(f'Synthetic 4D-STEM: {data.shape}')

Synthetic 4D-STEM: (32, 32, 96, 96)


In [14]:
w = Bin(data, pixel_size=pixel_size, k_pixel_size=k_pixel_size, device='cpu')
w

Bin(shape=(32, 32, 96, 96), bin=(1, 1, 1, 1), binned_shape=(32, 32, 96, 96), mode=mean, edge=crop, backend=torch, device=cpu)

In [15]:
# Access binned tensor for downstream analysis
binned = w.get_binned_data(copy=False)
print(type(binned), tuple(binned.shape))

<class 'torch.Tensor'> (32, 32, 96, 96)
